# About: コンテナイメージの更新

---

リバースプロキシ(httpd)コンテナのイメージを更新する。

## 概要

リバースプロキシコンテナのコンテナイメージを更新します。

![コンテナイメージの更新](images/moodle-155-01.png)

`httpd:2` など、同じ名前（タグ）でコンテナイメージの内容が更新されている場合も、`pull`によるイメージの再取得を行うことで、コンテナイメージが更新されます。

### グループ名の指定

このNotebookの操作対象となる UnitGroup名を指定します。

In [ ]:
# (例)
# ugroup_name = 'Moodle'

ugroup_name =

#### チェック

指定された `UnitGroup`名に対応する group_varsファイルが存在していることを確認します。エラーになる場合は、指定したUnitGroup名が正しくないか、「010-パラメータ設定.ipynb」のNotebookを実行していないことが考えられます。

In [ ]:
from pathlib import Path
if not (Path('group_vars') / (ugroup_name + '.yml')).exists():
    raise RuntimeError(f"ERROR: not exists {ugroup_name + '.yml'}")

UnitGroupに属する VCノードに対して Ansible で操作できることを確認します。

In [ ]:
!ansible {ugroup_name} -m ping
!ansible {ugroup_name} -b -a 'whoami'

### VCCアクセストークンの入力

VCCにアクセスするためのトークンを入力します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

#### チェック

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from vcpsdk.vcpsdk import VcpSDK
vcp = VcpSDK(vcc_access_token)

## 準備

「010-パラメータの設定.ipynb」などで既に設定したパラメータを引き継ぐための準備を行います。

group_varsファイルに保存されているパラメータを読み込みます。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

VCCのVault サーバにアクセスする際に必要となるパラメータを環境変数に設定します。

In [ ]:
import os
os.environ['VAULT_ADDR'] = vcp.vcc_info()['vault_url']
os.environ['VAULT_TOKEN'] = vcc_access_token

## パラメータの設定

新しいコンテナイメージ名を指定します。

更新後のリバースプロキシコンテナのイメージ名を指定してください。

In [ ]:
update_rproxy_image_name = 'httpd:2'

## アップデート環境の起動

リバースプロキシコンテナを新しいコンテナイメージで起動し直します。

### コンテナのタグづけ

後で差し戻しができるように、現在使用しているリバースプロキシのコンテナイメージにタグを付けます。

In [ ]:
from datetime import datetime

rproxy_tag = datetime.now().strftime("%Y%m%d%H%M%S%f")
!ansible {ugroup_name} -a \
    'docker tag {gvars["rproxy_image_name"]} local/proxy:{rproxy_tag}'

コンテナイメージのタグが作成されたことを確認します。

In [ ]:
!ansible {ugroup_name} -a 'docker images local/proxy:{rproxy_tag}'

### group_vars の更新

group_varsに記録されているリバースプロキシコンテナのイメージ名を変更します。また、コンテナイメージの差し戻しに必要となる情報を記録しておきます。

In [ ]:
gvars = load_group_vars(ugroup_name)

next_info = {
    'rproxy_image_name': update_rproxy_image_name,
}

previous_info = {
    'rproxy_image_name_orig': gvars['rproxy_image_name'],
    'rproxy_image_name': f'local/proxy:{rproxy_tag}',
}
if 'previous_info_list' not in gvars:
    gvars['previous_info_list'] = []
gvars['previous_info_list'].insert(0, previous_info)

update_group_vars(ugroup_name, previous_info_list=gvars['previous_info_list'], **next_info)
gvars = load_group_vars(ugroup_name)

### docker-compose.yml の更新

現在の `docker-compose.yml`の内容を確認します。

In [ ]:
!ansible {ugroup_name} -a 'cat /opt/moodle/docker-compose.yml'

新しい `docker-compose.yml`をVCノードに配置します。

In [ ]:
!ansible {ugroup_name} -m template \
    -a 'src=template/docker/compose/docker-compose.yml \
    dest=/opt/moodle backup=yes'

記述内容を更新した後の docker-compose.ymlを確認します。

In [ ]:
!ansible {ugroup_name} -a 'cat /opt/moodle/docker-compose.yml'

### イメージの取得

コンテナイメージの取得を行います。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    docker-compose pull proxy'

### リバースプロキシコンテナの更新

現在使用しているコンテナイメージを確認します。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    docker-compose images'

指定したコンテナイメージに更新します。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    docker-compose up -d'

更新後のコンテナイメージを確認します。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    docker-compose images'

## Moodle を利用できることを確認

Moodle環境にアクセスして正しく動作していることを確認します。

次のセルを実行するとMoodleのアドレスが表示されます。

In [ ]:
from IPython.core.display import HTML
HTML(u'<a href="{0}/login" target="_blank">{0}/login</a>'.format(gvars['moodle_url']))